# Mein Bericht

##### Dies ist meine Abgabe

In [3]:
import requests
from eliza import eliza
import os
import json
from wendel_util import file_update
import dialog_util as util
import emorec


In [3]:
#vaccinations = requests.get('https://api.corona-zahlen.org/vaccinations')

In [4]:
file_update()
vaccinations = open('vaccinations.json')
#output vom RKI fest gespeichert
vaccinations = json.load(vaccinations)

Up To Date


In [5]:
#pickle.dump(vaccinations.json(), open("vaccinations_data.pkl",  "wb" ) )

In [6]:
#output vom RKI fest gespeichert
#vaccinations = pickle.load( open( "vaccination_data.pkl", "rb" ))

In [5]:
util.init_google()

In [6]:
#everything as lower case and numbers are normalized, remove stopwords, lemmatization, stemming
def normalize(in_s):
    return in_s.lower()

#asr
def do_input():
    return util.speech_input()

#finds states and vaccines in input and keys and values in dictionary, wenn ein keyword gefunden wird, wird die Schleife beendet
def semantic(input_s):
    semantics = {'state':'', 'vaccine':'', 'answer':0} #data structure
    for key in util.states_d.keys():
        if key in input_s:
            semantics['state'] = util.states_d[key]
    for key in util.vaccines_d.keys():
        if key in input_s:
            semantics['vaccine'] = util.vaccines_d[key]
    return semantics

#data from RKI API, gets required information from data
##import requests
##vaccinations = requests.get('https://api.corona-zahlen.org/vaccinations')
#fills the answer, gets input from RKI,  fills semantics object
def data(semantics):
    s = semantics['state']
    v = semantics['vaccine']
    if s: #state given
        if s != 'DE':
            if v: #and vaccine given
                semantics['answer'] = vaccinations["data"]["states"][s]["vaccination"][v] #
            else: #all vaccines for state
                semantics['answer'] = vaccinations["data"]["states"][s]["vaccinated"]
        else:
            if v: #and vaccine given
                semantics['answer'] = vaccinations["data"]["vaccination"][v]
            else: #all vaccines in Germany
                semantics['answer'] = vaccinations["data"]["vaccinated"]
    else: #no state
        if v: #but vaccine
            semantics['answer'] = vaccinations["data"]["vaccination"][v]
        else: #nothing given
            semantics['answer'] = None
    return semantics

In [7]:
#gives results and output string, returns text
def output(semantics, inputs): #fügt gut oder schlecht ein
    ret = ''
    s = semantics['state']
    v = semantics['vaccine']
    a = semantics['answer']
    if s: #if state is given
        s = util.state_names[s]
        if v: #vaccine also given
            v = util.vaccine_names[v]
            ret = 'Die Impfungen für {} mit {} sind {}'.format(s, v, a) #formatting
        else: #all vaccines for states
            ret = 'Die Impfungen für {} sind {}'.format(s, a)
    else: #no state
        if v: #but vaccine
            v = util.vaccine_names[v]
            ret = 'Die Impfungen in Deutschland mit {} sind {}'.format(v, a)
        else: #nothing given
            #ret = 'Diese Frage kann ich leider nicht beantworten'
            ret = elz.respond(inputs) #output
    return ret
    
def do_error():
    return 'Bitte wählen Sie ein Bundesland um die Coronazahlen zu erhalten'

In [8]:
#load deutsch.txt (Text), eliza speeks german
def init_eliza():
    root = "C:/Users/elisa/Seminare_Python/S_dialogsystem"
    elz = eliza.Eliza() #Instanz von Eliza Klasse
    elz.load(root+"/eliza/deutsch.txt") #use german version of Eliza
    return elz

In [9]:
#speech synthesis
def output_s(text):
    print('output: '+text)
    util.tts(text)

In [22]:
emo_dict = {'happiness': 'freundlich', 'neutral': 'neutral', 'anger': 'ärgerlich', 'sadness': 'traurig', 'disgust': 'angeekelt', 'fear': 'ängstlich', 'boredom': 'gelangweilt'}

In [19]:
def dialogmanager():
    output_s("Frag was!")
    input_s = do_input() #do_input() wird aufgerufen, um sich input zu holen
    input_s = normalize(input_s)
    while input_s and input_s != 'fertig': #wenn input-string 'fertig'  ist, wird Schleife nicht ausgeführt
        emotion = emoRec.classify(filename)[0]
        emotion_g = emo_dict[emotion]
        output_s('Ich merke du bist '+emotion_g)
        semantics = semantic(input_s) #funktion gibt etwas zurück
        semantics = data(semantics)
        out_string = output(semantics, input_s)
        output_s(out_string)
        input_s = do_input()
        input_s = normalize(input_s)

In [24]:
filename = 'myfile.wav'
elz = init_eliza()
emoRec = emorec.EmoRec()
dialogmanager()



output: Frag was!
Transcript 0: fertig
